# Linear Classification

This notebook implements a linear Support Vector Machine from Scratch.

In [1]:
import pandas as pd
import numpy as np
from typing import Tuple

df_train = pd.read_csv('data/SPECT.train',header=None)
df_test = pd.read_csv('data/SPECT.test',header=None)

train = df_train.values
test = df_test.values

y_train = train[:,0]
X_train = train[:,1:]
y_test = test[:,0]
X_test = test[:,1:]

### The Hinge Loss Function

Given predictions from a model `preds` and ground truth labels `y` $\in \left\{-1, 1\right\} $, `loss` calculates the hinge loss, that is, $\max\left(0, 1-y \cdot \textbf{x}^T \textbf{w}\right)$.  
Moreover, it calculates the gradient of the hinge loss with respect to $\textbf{w}$. 
Recall that the hinge loss function with regularization is given as $\textbf{L}(\textbf{w}) = \sum_{i=1}^n \left[ \max \left(0, 1-y_i \textbf{x}_i^T \textbf{w}  \right) + \frac{\lambda}{n} \cdot \textbf{w}^T\textbf{w}\right]$

The gradient of the hinge loss w.r.t. $\textbf{w}$ for *a single sample* $x_i$ therefore is:  
$\nabla_{x_i} \textbf{L}(\textbf{w}) = \begin{cases}   \frac{2\lambda}{n} \textbf{w} && \text{if  } y_i \textbf{x}_i^T \textbf{w} > 1 \\ \frac{2\lambda}{n} \textbf{w} - y_i \textbf{x}_i && \text{if  } y_i \textbf{x}_i^T \textbf{w} < 1     \end{cases}$  

**Note** however, that in this implementation the regularization is taken care of separately (in the `reg` function). Therefore, the gradient that is returend by `loss` **does not contain** the term $\frac{2\lambda}{n} \textbf{w}$!

In [2]:
def loss(preds: np.ndarray, y: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """Calculates hinge loss on predictions preds and ground truth y"""
    l = np.max(
        np.hstack([
            np.zeros((len(y), 1)),
            (1-y*preds).reshape(-1, 1)
        ]), axis=1
    )

    g = l.copy()
    mask = g > 0
    g[mask] = -y[mask]
    g[~mask] = 0

    return l, g

In [3]:
# Example:
loss(
    preds = np.array([2, 0, -3, 4]),
    y = np.array([1, 1, -1, -1])
    )

(array([0., 1., 0., 5.]), array([ 0., -1.,  0.,  1.]))

### The $\mathcal{L}_2$ Regularizer

Value of $\mathcal{L}_2$-regularizer $r$ and its gradient $g$ at point $\textbf{w}$. This is the separate regularization term which – in this implementation - is not included in the loss function but handeled separately.


$$r = \frac{\lambda}{2} \textbf{w}^{T}\textbf{w}$$

$$g = \lambda \textbf{w}$$

In [4]:
def l2_reg(w: np.ndarray, lambda_: float) -> Tuple[np.ndarray, np.ndarray]:
    """Returns value of L2 regularizer and its gradient at point w."""
    r = lambda_ / 2 * np.dot(w.T, w)
    g = lambda_ * w

    return r, g

### The `learn_model` function

For a given $n\times m$ design matrix $\textbf{X}$ and binary class label $\textbf{y}$ `learn_model` learns the parameters $\textbf{w}$ of a linear classification model.
The class label has to be $\in \left \{-1,1 \right \}$. 
The trade-off parameter between the empirical loss and the regularizer is $\lambda > 0$. 

In [5]:
def learn_model(X: np.ndarray, y: np.ndarray, lambda_: float, verbose: int = 0) -> np.ndarray:
    """Learns parameters w of a linear classification model on design matrix X with class labels y (either -1 or 1)."""
    max_iter = 200
    eps  = 0.001
    alpha = 1

    # Random initialization
    w = np.random.randn(X.shape[1])  

    for i in range(max_iter):
        # Calculate current predictions
        curr_preds = np.dot(X, w)
        # Calculate current loss & loss gradient
        curr_loss, curr_loss_gradient = loss(curr_preds, y)
        
        if verbose >= 1:
            print (f'loss: {np.mean(curr_loss)}')
        
        # Compute value of regularizer & regularizer gradient
        regu, regu_gradient = l2_reg(w, lambda_)

        # Calculate the whole gradient (including loss AND regularizer)
        # As the loss gradient depends on/contains x, these are mutiplied,
        # regu_gradient is added as it does not contain/depend on x.
        gradient = np.dot(X.T, curr_loss_gradient) + regu_gradient 

        # Calculate step size. Alternatively: while L(w)_new > L(w): a /= 2
        if (i > 0):
            alpha = alpha * (
                (np.dot(gradient_old.T, gradient_old))
                /(np.dot((gradient_old - gradient).T, gradient_old))
            )

            if verbose >= 2:
                print(f"alpha = {alpha}")

        # Update weights
        w = w - alpha * gradient

        # Stop if improvement is marginal.
        if (np.linalg.norm(alpha * gradient) < eps):
            break

        gradient_old = gradient
    return w

### The Predict Function

Once the weights are learned, the weights and a design matrix $X$ can be passed to the predict function to make predictions. The output of the decision function $f_{\textbf{w}}(\textbf{x}) = \textbf{x}^T \textbf{w}$ is converted to a class label $\in \{-1, 1\}$ using the $sign$ function. Note that this assumes that the bias term is included in $\textbf{w}$ and an additional column of $1$s was appended to $\textbf{x}$.

In [6]:
def predict(w: np.ndarray, X: np.ndarray) -> np.ndarray:
    """Given already learned weights w, predicts class labels for design matrix X."""
    preds = np.matmul(X, w.T)
    
    # sign-function converts output of decision function to class label.
    preds[preds > 0] = 1
    preds[preds <= 0] = -1

    return preds

### Using the Model

In [7]:
from sklearn.metrics import confusion_matrix, roc_auc_score

# Map class labels from {0, 1} to {-1, 1}
y_train[y_train == 0] = -1

# Learn weights and predict
w = learn_model(X_train, y_train, lambda_=10, verbose=2)
preds = predict(w, X_train)

print(" === TRAIN Set Performance ===")
print(confusion_matrix(y_train, preds))
print(f"ROC AUC = {roc_auc_score(y_train, preds)}")



loss: 1.242184916850876
loss: 14.603246301131836
alpha = 0.09280655555218147
loss: 1.009377189241246
alpha = 0.09891724547921082
loss: 1.3902623933638396
alpha = 0.052192663612385524
loss: 1.2260218229050577
alpha = 0.03022791595572736
loss: 0.8191647047596419
alpha = 0.08225856902721411
loss: 1.8341849144752829
alpha = 0.024502363973239044
loss: 0.7492270941674827
alpha = 0.030394666608220152
loss: 0.7528214205198218
alpha = 0.014149500768185906
loss: 0.725587807128435
alpha = 0.02665963947547244
loss: 0.7614814158533855
alpha = 0.008303640046587466
loss: 0.7203614807911649
alpha = 0.00930299852488308
loss: 0.7180861009791238
alpha = 0.007134165440248881
loss: 0.7202939112305451
alpha = 0.005515693737464294
loss: 0.7190644798880989
alpha = 0.0033594981029090474
loss: 0.7196854075819689
alpha = 0.003516937803319199
loss: 0.7194327012663961
alpha = 0.09999999999999984
loss: 0.7212500000000001
alpha = 0.029539034551996125
loss: 0.8189686866058776
alpha = 0.008310989600355308
loss: 0.7232

In [8]:
# Map class labels from {0, 1} to {-1, 1}
y_test[y_test == 0] = -1
preds = predict(w, X_test)

print(" === TRAIN Set Performance ===")
print(confusion_matrix(y_test, preds))
print(f"ROC AUC = {roc_auc_score(y_test, preds)}")

=== TRAIN Set Performance ===
[[  8   7]
 [ 11 161]]
ROC AUC = 0.7346899224806202


#### Comparison with Vanilla Random Forsest

In [9]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
preds = rf.predict(X_train)

print("=== TRAIN Set Performance ===")
print(confusion_matrix(y_train, preds))
print(f"ROC AUC = {roc_auc_score(y_train, preds)}")

=== TRAIN Set Performance ===
[[40  0]
 [ 5 35]]
ROC AUC = 0.9375


In [10]:
print("=== TEST Set Performance ===")
preds = rf.predict(X_test)

print(confusion_matrix(y_test, preds))
print(f"ROC AUC = {roc_auc_score(y_test, preds)}")

=== TEST Set Performance ===
[[ 12   3]
 [ 39 133]]
ROC AUC = 0.7866279069767442
